In [5]:

import pandas as pd
from scipy.stats import zscore

In [6]:
wallet_df = pd.read_csv('3_wallet_data.csv')
wallet_df = wallet_df.dropna(subset=['adv_id'])
wallet_df['created_at'] = pd.to_datetime(wallet_df['created_at'])
wallet_df['date'] = wallet_df['created_at'].dt.date

In [7]:
wallet_df

,adv_id,transaction_type,value,currency,source_entity_type,created_at,date
0,d016c006-f927-48bc-987d-40a2f3f4648b,DEBIT,1.01,rupiyo_token,OFFER_REWARD,2024-10-21 15:48:09.797749+00:00,2024-10-21
1,d016c006-f927-48bc-987d-40a2f3f4648b,DEBIT,0.02,rupiyo_token,DAILY_CHECKIN,2024-10-21 15:48:22.378338+00:00,2024-10-21
2,d016c006-f927-48bc-987d-40a2f3f4648b,DEBIT,0.05,rupiyo_token,DAILY_CHECKIN,2024-10-22 02:52:52.695171+00:00,2024-10-22
3,d016c006-f927-48bc-987d-40a2f3f4648b,DEBIT,0.10,rupiyo_token,DAILY_CHECKIN,2024-10-23 03:08:36.180386+00:00,2024-10-23
4,ed359404-3d8a-4f3e-9d4e-38e6ef575254,DEBIT,1.01,rupiyo_token,OFFER_REWARD,2024-10-21 15:48:09.797749+00:00,2024-10-21
...,...,...,...,...,...,...,...
1048570,e34d3a11-d084-4a6c-8df9-10ca1ad1abe1,DEBIT,208.00,offerwall_325,OFFER_REWARD,2024-10-17 05:11:16.344721+00:00,2024-10-17
1048571,3113e39c-881d-421f-862f-7131aa82af22,DEBIT,208.00,offerwall_325,OFFER_REWARD,2024-10-17 05:11:16.344721+00:00,2024-10-17
1048572,f4d1a817-1d85-4dc6-aa64-92f1a7cb50fc,DEBIT,208.00,offerwall_325,OFFER_REWARD,2024-10-17 05:11:16.344721+00:00,2024-10-17
1048573,75057402-2843-4003-a682-ee831cf68c75,DEBIT,208.00,offerwall_325,OFFER_REWARD,2024-10-17 05:11:16.344721+00:00,2024-10-17


In [8]:
wallet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1047979 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column              Non-Null Count    Dtype              
---  ------              --------------    -----              
 0   adv_id              1047979 non-null  object             
 1   transaction_type    1047979 non-null  object             
 2   value               1047979 non-null  float64            
 3   currency            1047979 non-null  object             
 4   source_entity_type  1047979 non-null  object             
 5   created_at          1047979 non-null  datetime64[ns, UTC]
 6   date                1047979 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(5)
memory usage: 64.0+ MB


In [9]:
debit_df = wallet_df[wallet_df['transaction_type'] == 'DEBIT']
credit_df = wallet_df[wallet_df['transaction_type'] == 'CREDIT']

In [10]:
debit_stats = debit_df.groupby('adv_id').agg(
    total_debit_value=('value', 'sum'),
    debit_count=('value', 'count')
)

In [11]:
credit_stats = credit_df.groupby('adv_id').agg(
    total_credit_value=('value', 'sum'),
    credit_count=('value', 'count')
)

In [12]:
activity_stats = wallet_df.groupby('adv_id').agg(
    active_days=('date', lambda x: x.nunique()),
    total_txns=('value', 'count')
)

In [13]:
user_stats = debit_stats.join(credit_stats, how='outer').join(activity_stats, how='outer').fillna(0)


In [14]:
user_stats['txn_per_day'] = user_stats['total_txns'] / (user_stats['active_days'] + 1e-5)
user_stats['credit_debit_ratio'] = user_stats['total_credit_value'] / (user_stats['total_debit_value'] + 1e-5)

In [15]:
z_scores = user_stats[['total_debit_value', 'total_credit_value', 'debit_count', 'credit_count', 'txn_per_day', 'credit_debit_ratio']].apply(zscore)

In [16]:
anomalous_users = z_scores[(z_scores.abs() > 3).any(axis=1)]

In [17]:
print(f"Number of anomalous users: {len(anomalous_users)}")
print(anomalous_users.head())

Number of anomalous users: 3491
                                      total_debit_value  total_credit_value  \
adv_id                                                                        
0000-0000                                      0.105740                 NaN   
00000000-0000-0000-0000-000000000000           0.480245                 NaN   
000aa241-f1d9-45ed-8f6d-980100059643           0.146295                 NaN   
00111b7f-5eb1-4713-804c-14903398e1ba           1.302704                 NaN   
00339774-b95f-4c78-9639-7566f33c01ba          -0.075609                 NaN   

                                      debit_count  credit_count  txn_per_day  \
adv_id                                                                         
0000-0000                                6.345583           NaN     4.153366   
00000000-0000-0000-0000-000000000000    67.758775           NaN    49.587964   
000aa241-f1d9-45ed-8f6d-980100059643     3.104781           NaN     2.149653   
00111b7f-5eb1-

# Assumptions Made
Only valid user IDs (adv_id) are analyzed.

All transaction types and currencies are considered; no filtering applied.

Outliers are detected using Z-score threshold > 3 or < -3.

Anomalies indicate unusual behavior, potentially fraudulent.

CREDIT = user pays GreedyGame, DEBIT = user earns from GreedyGame.

Transaction frequency is based on total transactions / active days.

No time zone adjustments assumed in timestamps.